Création d'un cluster local

In [ ]:
import dask.distributed

cluster = dask.distributed.LocalCluster()

Connexion d'un client au cluster

In [ ]:
import dask.distributed

client = dask.distributed.Client(cluster)
client

# 1 - Image processing with Dask

First some imports

In [ ]:
import dask.array as da
import dask_image.imread
import dask_image.ndfilters
import dask_image.ndmeasure

We'll also use matplotlib to display image results in this notebook

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

## Getting the example data

We'll use some example image data from the scikit-image library in this tutorial. These images are very small, but will allow us to demonstrate the functionality of dask-image. <br>
Let's download and save a public domain image of the astronaut Eileen Collins to a temporary directory. This image was originally downloaded from the NASA Great Images database https://flic.kr/p/r9qvLn, but we'll access it with scikit-image's data.astronaut() method.

In [ ]:
!mkdir -p temp

In [ ]:
import os

from skimage import data, io

output_filename = os.path.join("temp", "astronaut.png")
io.imsave(output_filename, data.astronaut())

Really large datasets often can't fit all of the data into a single file, so we'll chop this image into four parts and save the image tiles to a second temporary directory. This will give you a better idea of how you can use dask-image on a real dataset.

In [ ]:
!mkdir -p temp-tiles

In [ ]:
io.imsave(
    os.path.join("temp-tiles", "image-00.png"), data.astronaut()[:256, :256, :]
)  # top left
io.imsave(
    os.path.join("temp-tiles", "image-01.png"), data.astronaut()[:256, 256:, :]
)  # top right
io.imsave(
    os.path.join("temp-tiles", "image-10.png"), data.astronaut()[256:, :256, :]
)  # bottom left
io.imsave(
    os.path.join("temp-tiles", "image-11.png"), data.astronaut()[256:, 256:, :]
)  # bottom right

Now we have some data saved, let's practise reading in files with dask-image and processing our images.

## Reading in image data

Reading a single image. <br>
Let's load a public domain image of the astronaut Eileen Collins with dask-image imread(). This image was originally downloaded from the NASA Great Images database https://flic.kr/p/r9qvLn.

In [ ]:
import os

filename = os.path.join("temp", "astronaut.png")
print(filename)

In [ ]:
astronaut = dask_image.imread.imread(filename)
astronaut

In [ ]:
plt.imshow(astronaut[0, ...])  # display the first (and only) frame of the image

This has created a dask array with shape=(1, 512, 512, 3). This means it contains one image frame with 512 rows, 512 columns, and 3 color channels.
Since the image is relatively small, it fits entirely within one dask-image chunk, with chunksize=(1, 512, 512, 3).

## Reading multiple images

In many cases, you may have multiple images stored on disk, for example: image_00.png, image_01.png, ... image_NN.png. These can be read into a dask array as multiple image frames. <br>

Here we have the astronaut image split into four non-overlapping tiles: <br>

- image_00.png = top left image (index 0,0)
- image_01.png = top right image (index 0,1)
- image_10.png = bottom left image (index 1,0)
- image_11.png = bottom right image (index 1,1)

This filename pattern can be matched with regex: image-*.png

In [ ]:
!ls temp-tiles

In [ ]:
import os

filename_pattern = os.path.join("temp-tiles", "image-*.png")
tiled_astronaut_images = dask_image.imread.imread(filename_pattern)
tiled_astronaut_images

This has created a dask array with shape=(4, 256, 256, 3). This means it contains four image frames; each with 256 rows, 256 columns, and 3 color channels.
There are four chunks in this particular case. Each image frame here is a separate chunk with chunksize=(1, 256, 256, 3).

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2)
ax[0, 0].imshow(tiled_astronaut_images[0])
ax[0, 1].imshow(tiled_astronaut_images[1])
ax[1, 0].imshow(tiled_astronaut_images[2])
ax[1, 1].imshow(tiled_astronaut_images[3])
plt.show()

## Applying your own custom function to images

Next you'll want to do some image processing, and apply a function to your images. <br>

We'll use a very simple example: converting an RGB image to grayscale. But you can also use this method to apply arbittrary functions to dask images. To convert our image to grayscale, we'll use the equation to calculate luminance: <br>

> - Y = 0.2125 R + 0.7154 G + 0.0721 B

We'll write the function for this equation as follows **exercice : write the function**

In [ ]:
def grayscale(rgb):
    result = ...  # TODO
    return result

**Solution 1** at the end

Let's apply this function to the astronaut image we read in as a single file and visualize the computation graph.

(Visualizing the computation graph isn't necessary most of the time but it's helpful to know what dask is doing under the hood, and it can also be very useful for debugging problems.)

In [ ]:
single_image_result = grayscale(astronaut)
single_image_result

In [ ]:
single_image_result.visualize()

We also see that there are no longer three color channels in the shape of the result, and that the output image is as expected.

In [ ]:
print("Original image dimensions: ", astronaut.shape)
print("Processed image dimensions:", single_image_result.shape)

fig, (ax0, ax1) = plt.subplots(nrows=1, ncols=2)
ax0.imshow(astronaut[0, ...])  # display the first (and only) frame of the image
ax1.imshow(
    single_image_result[0, ...], cmap="gray"
)  # display the first (and only) frame of the image

# Subplot headings
ax0.set_title("Original image")
ax1.set_title("Processed image")

# Don't display axes
ax0.axis("off")
ax1.axis("off")

# Display images
plt.show(fig)

## Embarrassingly parallel problems

The syntax is identical to apply a function to multiple images or dask chunks. This is an example of an embarrassingly parallel problem, and we see that dask automatically creates a computation graph for each chunk.

In [ ]:
result = ...  # TODO

**Solution 2** at the end

In [ ]:
result.visualize()

Let's take a look at the results.

In [ ]:
fig, ((ax0, ax1), (ax2, ax3)) = plt.subplots(nrows=2, ncols=2)
ax0.imshow(result[0, ...], cmap="gray")
ax1.imshow(result[1, ...], cmap="gray")
ax2.imshow(result[2, ...], cmap="gray")
ax3.imshow(result[3, ...], cmap="gray")

# Subplot headings
ax0.set_title("First chunk")
ax1.set_title("Second chunk")
ax2.set_title("Thurd chunk")
ax3.set_title("Fourth chunk")

# Don't display axes
ax0.axis("off")
ax1.axis("off")
ax2.axis("off")
ax3.axis("off")

# Display images
plt.show(fig)

## Joining partial images together

OK, Things are looking pretty good! But how can we join these image chunks together?

So far, we haven't needed any information from neighboring pixels to do our calculations. But there are lots of functions (like those in dask-image ndfilters) that do need this for accurate results. You could end up with unwanted edge effects if you don't tell dask how your images should be joined.

Dask has several ways to join chunks together: Stack, Concatenate, and Block.

Block is very versatile, so we'll use that in this next example. You simply pass in a list (or list of lists) to tell dask the spatial relationship between image chunks.

In [ ]:
data = [[result[0, ...], result[1, ...]], [result[2, ...], result[3, ...]]]
combined_image = da.block(data)
combined_image.shape

In [ ]:
combined_image

In [ ]:
plt.imshow(combined_image, cmap="gray")
plt.show()

## A segmentation analysis pipeline

We'll walk through a simple image segmentation and analysis pipeline with three steps:

1. Filtering
2. Segmenting
3. Analyzing

## Filtering
Most analysis pipelines require some degree of image preprocessing. dask-image has a number of inbuilt filters available via dask-image ndfilters.

Commonly a guassian filter may used to smooth the image before segmentation. This causes some loss of sharpness in the image, but can improve segmentation quality for methods that rely on image thresholding.

In [ ]:
smoothed_image = (
    ...
)  # TODO apply gaussian filter from dask_image.ndfilters with unit variance

**solution 3 at the end**

We see a small amount of blur in the smoothed image.

In [ ]:
fig, (ax0, ax1) = plt.subplots(nrows=1, ncols=2)
ax0.imshow(smoothed_image, cmap="gray")
ax1.imshow(smoothed_image - combined_image, cmap="gray")

# Subplot headings
ax0.set_title("Smoothed image")
ax1.set_title("Difference from original")

# Don't display axes
ax0.axis("off")
ax1.axis("off")

# Display images
plt.show(fig)

Since the gaussian filter uses information from neighbouring pixels, the computational graph looks more complicated than the ones we looked at earlier. This is no longer embarrassingly parallel. Where possible dask keeps the computations for each of the four image chunks separate, but must combine information from different chunks near the edges.

In [ ]:
smoothed_image.visualize()

## Segmenting
After the image preprocessing, we segment regions of interest from the data. We'll use a simple arbitrary threshold as the cutoff, at 75% of the maximum intensity of the smoothed image.

In [ ]:
threshold_value = 0.75 * da.max(smoothed_image).compute()
print(threshold_value)

In [ ]:
threshold_image = smoothed_image > threshold_value
plt.imshow(threshold_image, cmap="gray")

Next, we label each region of connected pixels above the threshold value. For this we use the label function from dask-image ndmeasure. This will return both the label image, and the number of labels.

In [ ]:
label_image, num_labels = ...  # TODO

**Solution 4** at the end

In [ ]:
print("Number of labels:", int(num_labels))  # You have to find 78 labels
plt.imshow(label_image, cmap="viridis")
plt.show()

# 2 - Reading and manipulating tiled GeoTIFF datasets

This notebook shows how to perform simple calculations with a GeoTIFF dataset using XArray and Dask. 

We load and rescale a time series of Pléiades HR images and compute NDVI (Normalized difference vegetation index). This can be used to distinguish green vegetation from areas of bare land or water.

We'll use a timeseries from Toulouse

In [ ]:
import json
import os
from pathlib import Path

import matplotlib.pyplot as plt
import rasterio
import requests

%matplotlib inline

## Download data

In [ ]:
image_urls = [
    "https://storage.googleapis.com/fchouteau-isae-otsu/IMG_PHR1A_PMS_201504131053028_small.tif",
    "https://storage.googleapis.com/fchouteau-isae-otsu/IMG_PHR1A_PMS_201506041053076_small.tif",
    "https://storage.googleapis.com/fchouteau-isae-otsu/IMG_PHR1A_PMS_201609071056565_small.tif",
    "https://storage.googleapis.com/fchouteau-isae-otsu/IMG_PHR1A_PMS_201707041049221_small.tif",
    "https://storage.googleapis.com/fchouteau-isae-otsu/IMG_PHR1A_PMS_201908041045469_small.tif",
]

In [ ]:
def download_file(in_url: str, out_filename: Path):
    if not (out_filename).exists():
        out_filename.parent.mkdir(exist_ok=True)
        print("Downloading", in_url)
        response = requests.get(in_url)
        with open(out_filename, "wb") as f:
            f.write(response.content)

In [ ]:
for image_url in image_urls:
    download_file(image_url, Path() / "data" / image_url.split("/")[-1])

## Check image metadata

Let's see if the image is tiled so we can select a chunk size.

In [ ]:
img = rasterio.open("data/IMG_PHR1A_PMS_201908041045469_small.tif")
print(img.is_tiled)

In [ ]:
img.block_shapes

The image has separate blocks for each band with block size 512 x 512. 

## Create xarray datasets

In [ ]:
import datetime

import dask.array as da
import numpy as np
import rioxarray
import xarray as xr

In [ ]:
image_files = sorted(list(Path("data").glob("*.tif")))
print(image_files)

In [ ]:
def path_to_datetime(image_file):
    date = image_file.stem
    date = date.split("_")[-2][:8]
    date = datetime.datetime.strptime(date, "%Y%m%d")
    return date

In [ ]:
timestamps = []
timeseries = []

for image_file in image_files:
    timestamps.append(path_to_datetime(image_file))
    img = rioxarray.open_rasterio(image_file, chunks=512)
    img = img.assign_coords(coords=dict(band=["RED", "GREEN", "BLUE", "NIR"]))
    timeseries.append(img)

time_var = xr.Variable("time", timestamps)

timeseries = xr.concat(timeseries, dim=time_var)

In [ ]:
timeseries

In [ ]:
timeseries[0]

## Calculate and display the image in RGB 8 bits

The original image dynamic is in 12 bits (0-4095) in uint16, convert it to 8bits (0-255)

In [ ]:
def compute_display_chunk(tile):
    img = ...
    return img

**Solution 5** at the end

In [ ]:
display_timeseries = compute_display_chunk(timeseries)

In [ ]:
display_timeseries

In [ ]:
display_timeseries.data.visualize()

In [ ]:
display_timeseries = display_timeseries.compute().transpose("time", "y", "x", "band")

In [ ]:
for display_image in display_timeseries:
    plt.title(display_image.time.data)
    plt.imshow(display_image)
    plt.show()

## Calculate and display NDVI

Now that we have the image as reflectance values, we are ready to compute NDVI.

$$
NDVI = \frac{NIR - Red}{NIR + Red}
$$

This highlights areas of healthy vegetation with high NDVI values, which appear as green in the image below.

Hint : Don't forget to convert the type of your images from 16bits to something that can go negative

In [ ]:
def compute_ndvi_chunk(tile):
    r = ...
    n = ...

    ndvi = ...

    return ndvi

**Solution 6** at the end

In [ ]:
ndvi_timeseries = compute_ndvi_chunk(timeseries)

In [ ]:
ndvi_timeseries

In [ ]:
type(ndvi_timeseries.data)

In [ ]:
ndvi_timeseries.data.visualize(optimize_graph=False)

In [ ]:
ndvi_timeseries = ndvi_timeseries.compute()

In [ ]:
for ndvi_image in ndvi_timeseries:
    plt.title(ndvi_image.time.data)
    plt.imshow(ndvi_image, cmap="viridis")
    plt.show()

# 3 - Cleanup

In [ ]:
cluster.shutdown()

In [ ]:
client.close()

# 4 - Solutions

**Solution 1**

```python
def grayscale(rgb):
    result = (rgb[..., 0] * 0.2125) + (rgb[..., 1] * 0.7154) + (rgb[..., 2] * 0.0721)
    return result
```

**Solution 2**

```python
result = grayscale(tiled_astronaut_images)
result
```

**Solution 3**

```python
smoothed_image = dask_image.ndfilters.gaussian_filter(combined_image, sigma=[1, 1])
```

**Solution 4**

```python
label_image, num_labels = dask_image.ndmeasure.label(threshold_image)
```

**Solution 5**

```python
def compute_display_chunk(tile):
    img = (
        tile[:, [0, 1, 2], :, :].clip(0, 4095).astype(np.float32) / 4095.0 * 255.0
    ).astype(np.uint8)
    return img
```

**Solution 6**

```python
def compute_ndvi_chunk(tile):
    r = tile[:, 0, :, :].clip(0, 4095).astype(np.float32) / 4095.0
    n = tile[:, 3, :, :].clip(0, 4095).astype(np.float32) / 4095.0

    ndvi = n - r / n + r

    return ndvi
```